<a href="https://colab.research.google.com/github/vivianlin2000/Computer-Vision-Classification-for-Chest-X-Ray-Imaging/blob/main/Frangi_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import ast
import numpy as np 
import pandas as pd
import plotly.graph_objects as go 
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from skimage import filters
from skimage import exposure
from skimage import io, color
from skimage.transform import resize
from skimage.io import imsave, imread
from sklearn.decomposition import FastICA
!pip install python-gdcm
!pip install pydicom
import pydicom as pyd
from PIL import Image
# split into train and test set
from sklearn.model_selection import train_test_split

#############################################################################
study_df = pd.read_csv('./data/train_study_level.csv', index_col=None)
image_df = pd.read_csv('./data/train_image_level.csv', index_col=None)
#############################################################################
X_train, X_test, y_train, y_test = train_test_split(study_df['id'], study_df[study_df.columns[1:]], test_size=0.2, random_state=42)
print('# of train samples:', len(X_train))
print('# of test samples:', len(X_test))
#############################################################################
# create image dictionary 
# {study_id: [image_ids]}

study_img_dict = {}
for study_id in study_df['id']:
    img_ids = image_df[image_df['StudyInstanceUID'] == study_id]['id']
    study_img_dict[study_id] = img_ids

study_img_dict
#############################################################################
# iterate through studies
for study_id in X_train:
    count = 0
    # iterate through study's images
    for img_id in study_img_dict[study_id]:
        # get the file
        # check if file exists
        if img_id not in file_dict.keys():
            break
        # read file
        dicom = pyd.read_file(file_dict[img_id], stop_before_pixels=False)
        img = dicom.pixel_array
        # resize image for faster processing
        x_image = resize(img, (512, 512), anti_aliasing=True)

### Frangi Filter Pseudocode


Inputs/params: 

image : (NxM) ndarray
- input image array

sigmas : iterable of floats
- scales of filter

alpha : float
- frangi correction constant that adjusts the filter's sensitivity to deviation from a plate-like structure

beta : float
- frangi correction constant that adjusts the filter's sensitivity to deviation from a blob-like structure

c : float
- frangi correction constant that adjusts the filter's sensitivity to areas of high variance/texture/structure

black_ridges : boolean
- True if black ridges; False if white ridges.

mode : {'constant', 'reflect', 'wrap', 'nearest', 'mirror'}
- method of handling values outside image borders

cval : float
- Used in conjunction with mode 'constant', the value outside the image boundaries.

```
Frangi Filter Pseudocode:

#get data/image into desirable format
#generalizable to 3d, for our purposes we will focus on 2d
check black ridges
check constraints of inputs (image dimensions -- 2d or 3d array, nonnegative sigma values)

#filter along all sigma
for sigma in sigmas:
  hessian <- hessian_matrix(image, sigma=sigma) #hessian at scale sigma
  hessian_eigenvalues <- hessian_matrix_eigvals(hessian)
  hessian_eigenvalues <- sort_by_abs(hessian eigenvalues)

  # compute sensitivity to deviation from a plate-like structure eq11,15
  # (3d) r_a <- lambda2/lambda3

  # compute sensitivity to deviation from a blob-like structure, eq10,15
  # (3d) r_b <- lambda1/sqrt(lambda2 * lambda3)
  r_b <- lambda1/lambda2

  # compute sensitivity to areas of high variance/texture/structure, eq12
  # (3d) s <- sqrt(lambda1**2 + lambda2**2 + lambda3**2)
  s <- sqrt(lambda1**2 + lambda2**2)

  # output image
  for each pixel in image:
    filter <- (np.exp(-r_b**2/beta**2)*(1 - np.exp(-s/c**2)))

# Remove background
filtered_arr[lambdas_arr > 0] <- 0
# Return for every pixel the maximum value over all (sigma) scales (less noisy)


```






---